In [ ]:
from nitropulse import RismaData

In [ ]:
risma = RismaData(workspace_dir='/home/morteza/.nitropulse')

In [ ]:
risma_stations = ['RISMA_MB1', 'RISMA_MB2', 
                  'RISMA_MB3', 'RISMA_MB4', 
                  'RISMA_MB5', 'RISMA_MB6', 
                  'RISMA_MB7', 'RISMA_MB8', 
                  'RISMA_MB9', 'RISMA_MB10', 
                  'RISMA_MB11', 'RISMA_MB12', 
                  'RISMA_MB13', 'RISMA_MB14', 
                  'RISMA_MB15', 'RISMA_MB16',
                  'RISMA_MB17', 'RISMA_MB18', 
                  'RISMA_MB19', 'RISMA_MB20',
                  'RISMA_MB21', 'RISMA_MB22', 
                  'RISMA_MB23', 'RISMA_MB24',
                  'RISMA_MB25', 'RISMA_MB26',
                  'RISMA_SK1', 'RISMA_SK2', 
                  'RISMA_SK3', 'RISMA_SK4',

                  ]
params = ['Air Temp', 'Soil temperature', 'Soil Moisture']
sensors= 'average'
depths = ['0 to 5 cm', '5 cm']

In [ ]:
# risma.download_risma_data(
#     out_dir='../assets/inputs/RISMA_CSV_files', 
#     stations=risma_stations, parameters=params, 
#     sensors=sensors, depths=depths, 
#     start_date='2010-01-01', end_date='2024-01-01')

In [ ]:
risma_df = risma.load_df(depth='0 to 5 cm')
print(risma_df.shape)
risma_df.head()

In [ ]:
# risma_df.to_csv("/home/morteza/.nitropulse/outputs/risma_df.csv", index=False)

In [1]:
import pandas as pd

risma_df = pd.read_csv('~/.nitropulse/outputs/risma_df.csv')
risma_df.head()

,date,station,depth,mean_sst,mean_airt,airt,prcp,ssm,sst,year,doy,depth_cm,bulk,sand,silt,clay,classification
0,2013-04-16 18:00:00,MB1,0 to 5 cm,0.108333,-1.52,-1.82,0.0,-0.160000,0.000000,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam
1,2013-04-16 18:15:00,MB1,0 to 5 cm,0.108333,-1.52,-2.02,0.0,0.206667,0.050000,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam
2,2013-04-16 18:30:00,MB1,0 to 5 cm,0.108333,-1.52,-1.89,0.0,0.206667,0.050000,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam
3,2013-04-16 18:45:00,MB1,0 to 5 cm,0.108333,-1.52,-1.67,0.0,0.208333,0.083333,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam
4,2013-04-16 19:00:00,MB1,0 to 5 cm,0.108333,-1.52,-1.72,0.0,0.208333,0.083333,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam


In [5]:
# count records for station MB15 (matches MB15, MB_15 or MB-15) in year 2015
mask_station = risma_df['station'].astype(str).str.contains(r'\bMB[_-]?3\b', case=False, na=False)
mask_year = risma_df['year'] == 2023
sub_df = risma_df.loc[mask_station & mask_year]
sub_df.head()

,date,station,depth,mean_sst,mean_airt,airt,prcp,ssm,sst,year,doy,depth_cm,bulk,sand,silt,clay,classification
6597056,2023-01-01 00:00:00,MB3,0 to 5 cm,-4.875,-8.315,-8.30,0.0,0.116667,-5.016667,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam
6597163,2023-01-01 01:15:00,MB3,0 to 5 cm,-4.875,-8.315,-8.68,0.0,0.116667,-5.033333,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam
6597194,2023-01-01 01:30:00,MB3,0 to 5 cm,-4.875,-8.315,-8.72,0.0,0.116667,-5.050000,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam
6597225,2023-01-01 01:45:00,MB3,0 to 5 cm,-4.875,-8.315,-8.78,0.0,0.116667,-5.050000,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam
6597256,2023-01-01 02:00:00,MB3,0 to 5 cm,-4.875,-8.315,-8.86,0.0,0.115000,-5.083333,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam


In [6]:
# aggregate to daily (sum precipitation, mean SSM) so they can be plotted together
sub_df_day = sub_df.copy()
sub_df_day["date"] = pd.to_datetime(sub_df_day["date"]).dt.floor("D")

precip_daily = (
    sub_df_day.groupby("date", as_index=False)["prcp"]
    .sum()
    .rename(columns={"prcp": "prcp_daily"})
)
sst_daily = (
    sub_df_day.groupby("date", as_index=False)["sst"]
    .mean()
    .rename(columns={"sst": "sst_daily"})
)

# merged table for easy plotting/inspection
daily = pd.merge(sst_daily, precip_daily, on="date", how="outer").sort_values("date").reset_index(drop=True)
daily["prcp_daily"] = daily["prcp_daily"].fillna(0)
daily["station"] = "MB15"

daily.head()

,date,sst_daily,prcp_daily,station
0,2023-01-01,-4.967210,0.0,MB15
1,2023-01-02,-5.436458,0.0,MB15
2,2023-01-03,-7.083333,0.0,MB15
3,2023-01-04,-7.775521,0.0,MB15
4,2023-01-05,-8.280729,0.0,MB15


In [7]:
import plotly.express as px

fig = px.line(sub_df, x='date', y='ssm', color='station', title='Soil Moisture', line_shape='linear')
fig.update_layout(
    title={
        'text': 'Soil Moisture at RISMA_MB15 (2015)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    legend_title_text='Station',
    font=dict(
        family="Arial",
        size=12,
        color="Black"
    )
)
# add precipitation as a secondary y-axis (bars) to the existing figure
fig.add_bar(x=daily['date'], y=daily['prcp_daily'], name='Precipitation', yaxis='y2', opacity=0.75)
fig.update_layout(
    yaxis2=dict(
        title='Precipitation (mm)',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    bargap=0
)

# shade periods where daily surface temp (sst_daily) <= 0°C to indicate frozen soil
frozen = sst_daily.sort_values("date").reset_index(drop=True)
frozen["is_frozen"] = frozen["sst_daily"] <= 0

# compute contiguous frozen ranges
ranges = []
in_range = False
start = None
for i, row in frozen.iterrows():
    if row["is_frozen"] and not in_range:
        in_range = True
        start = row["date"]
    elif not row["is_frozen"] and in_range:
        end = frozen.loc[i - 1, "date"]
        ranges.append((start, end))
        in_range = False
if in_range:
    ranges.append((start, frozen.loc[frozen.index[-1], "date"]))

# add semi-transparent background rectangles for each frozen period
for start, end in ranges:
    fig.add_shape(
        type="rect",
        xref="x",
        yref="paper",
        x0=start,
        x1=end,
        y0=0,
        y1=1,
        fillcolor="LightSkyBlue",
        opacity=0.25,
        layer="below",
        line_width=0,
    )

# add a small annotation to explain the shading
fig.update_layout(
    annotations=[
        dict(
            xref="paper",
            x=0.01,
            y=0.4,
            xanchor="left",
            yanchor="top",
            text="Shaded = frozen soil (sst ≤ 0 °C)",
            showarrow=False,
            bgcolor="rgba(255,255,255,0.7)",
            bordercolor="LightSkyBlue",
        )
    ]
)

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Soil Moisture (m³/m³ VWC)", secondary_y=False)
fig.update_yaxes(title_text="Precipitation (mm/day)", secondary_y=True)

fig.show()